In [2]:
import pandas as pd
import numpy as np
import requests
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
url1 = 'https://ufal.mff.cuni.cz/~pecina/courses/npfl067/data/TEXTEN1.txt'
url2 = 'https://ufal.mff.cuni.cz/~pecina/courses/npfl067/data/TEXTCZ1.txt'
url3 = 'https://ufal.mff.cuni.cz/~pecina/courses/npfl067/data/TEXTEN1.ptg'
url4 = 'https://ufal.mff.cuni.cz/~pecina/courses/npfl067/data/TEXTCZ1.ptg'

## **1) Best friends**

In [ ]:
def get_data(url):
  " returns list of worlds and list of non frequent data"

  data = requests.get(url)
  data.encoding = 'iso-8859-2'
  data = data.text.split('\n')[:-1]

  # find frequent words
  vocab,counts = np.unique(data, return_counts=True)
  non_frequent = vocab[counts < 10]

  return data, non_frequent

In [ ]:
def put_to_dict(key, dc):
  # put key in dictionary
  if  key not in dc:
    dc[key] = 1
  else:
    dc[key] +=1

def compute_counts(data, distance):
  # compute count of bigrams and unigrams
  text_length = len(data)
  pairs_counts = dict()
  word_counts = dict()

  for i in range(text_length-1):

    if distance == 50:
      # for distant pairs
      lb = max(0, i-distance)
      up = min(text_length, i+distance +1)
      word1 = data[i]
      put_to_dict(word1, word_counts)

      for j in range(lb, up):
        if j == i or j == i+1 or j == i-1 : continue
        word2 = data[j]
        put_to_dict((word1, word2), pairs_counts)

    else:
      # for non-distant bigrams
      word1 = data[i]
      word2 = data[i+1]

      put_to_dict((word1, word2), pairs_counts)
      put_to_dict(word1, word_counts)

  put_to_dict(data[-1], word_counts)

  return word_counts, pairs_counts

In [ ]:
def compute_PMI(unigram_c, bigram_c):
  # compute PMI based on bigram and unigram distributions for bigrams from which both words apperars >= 10 times

  PMI = dict()
  N_u = sum(unigram_c.values())
  N_b = sum(bigram_c.values())

  for pair,count in bigram_c.items():
    word1, word2 = pair
    if unigram_c[word1] < 10 or unigram_c[word2] < 10: continue

    joint_prob = count/N_b
    PMI[pair] = math.log(joint_prob / (unigram_c[word1]/N_u * unigram_c[word2]/N_u), 2)
  return PMI

In [ ]:
def compute_pmi_for_dataset(dataset,distance):
  # returns dataframe for pmi and bigrams obtained from dataset
  unigram_p, joint_p = compute_counts(dataset, distance)
  pmi = compute_PMI(unigram_p, joint_p)
  df = pd.DataFrame(list(pmi.items()), columns=['pair', 'PMI'])
  return df


In [ ]:
# load data
data_CZ, non_freq_CZ = get_data(url2)
data_EN, non_freq_EN = get_data(url1)

In [ ]:
# compute PMI for czech dataset
df_bigrams_CZ = compute_pmi_for_dataset(data_CZ, 2)

In [ ]:
# show 10 pairs with negaitve PMI
df_bigrams_CZ[df_bigrams_CZ['PMI']<0][:10]

,pair,PMI
22,"(začala, ,)",-0.310172
48,"(,, členů)",-0.688684
76,"(,, v)",-0.369542
83,"(LN, .)",-1.105118
93,"(a, že)",-0.008790
134,"(., a)",-2.119988
137,"(., letech)",-0.193345
139,"(., E)",-0.419227
142,"("", v)",-0.865643
150,"(komise, a)",-0.275223


In [ ]:
# show best 20 pairs for czech dataset
df_bigrams_CZ.sort_values(by = 'PMI', ascending=False)[:20]

,pair,PMI
7251,"(Hamburger, SV)",14.288950
2349,"(Los, Angeles)",14.062442
16869,"(Johna, Newcomba)",13.762882
17205,"(Č., Budějovice)",13.633599
24770,"(série, ATP)",13.468968
24874,"(turnajové, série)",13.434411
3811,"(Tomáš, Ježek)",13.428981
8449,"(Lidové, noviny)",13.329922
8862,"(Lidových, novin)",13.271028
2705,"(veřejného, mínění)",13.062442


In [ ]:
# compute pmi for distant pairs czech dataset
df_distant_CZ = compute_pmi_for_dataset(data_CZ, 50)

In [ ]:
df_distant_CZ.sort_values(by = 'PMI', ascending=False)[:20]

,pair,PMI
994100,"(výher, výher)",9.855555
714065,"(žel, žel)",9.136365
294204,"(13h, 13h)",8.855555
640194,"(Sandžaku, Sandžaku)",8.826409
1381966,"(Petrof, Petrof)",8.785767
543726,"(Bělehrad, Benfica)",8.688906
543860,"(Benfica, Bělehrad)",8.688906
212728,"(CIA, CIA)",8.504481
227073,"(IFS, IFS)",8.437844
543861,"(Benfica, Atény)",8.340982


In [ ]:
# bigrams english dataset
df_bigrams_EN = compute_pmi_for_dataset(data_EN, 2)
df_bigrams_EN.sort_values(by = 'PMI', ascending=False)[:20]

,pair,PMI
9935,"(La, Plata)",14.169370
8263,"(Asa, Gray)",14.031867
7500,"(Fritz, Muller)",13.362016
2337,"(worth, while)",13.332869
3736,"(faced, tumbler)",13.262480
8953,"(lowly, organised)",13.216899
191,"(Malay, Archipelago)",13.110477
19636,"(shoulder, stripe)",13.053893
3523,"(Great, Britain)",12.914557
6782,"(United, States)",12.847442


In [ ]:
# pmi for distant pairs english dataset
df_distant_EN = compute_pmi_for_dataset(data_EN, 50)
df_distant_EN.sort_values(by = 'PMI', ascending=False)[:20]

,pair,PMI
1189784,"(floated, dried)",8.692331
1189935,"(dried, floated)",8.692331
1190282,"(dried, dried)",8.303288
1189651,"(floated, floated)",8.192257
1190392,"(germinated, dried)",8.165785
1190286,"(dried, germinated)",8.165785
375380,"(heath, heath)",8.060838
384080,"(clover, clover)",8.026448
1190387,"(germinated, floated)",7.969865
1190248,"(floated, germinated)",7.969865


#2) Word classes


In [3]:
def get_data(url):
  # returns list of words and tags of data

  data = requests.get(url)
  data.encoding = 'iso-8859-2'
  lines = data.text.split('\n')[:-1]
  words,tags = [], []
  for line in lines:
    word, tag = line.split('/')
    words.append(word)
    tags.append(tag)

  return words, tags

In [4]:
def compute_counts(words):
  # compute bigram and unigram counts

  bigram_counts = dict()
  unigram_counts = dict()

  for i in range(len(words) - 1):
      w1, w2 = words[i], words[i + 1]

      bigram_counts[(w1, w2)] = bigram_counts.get((w1, w2), 0) + 1
      unigram_counts[w1] = unigram_counts.get(w1, 0) + 1

  unigram_counts[words[-1]] = unigram_counts.get(words[-1], 0) + 1

  return unigram_counts, bigram_counts


In [5]:
def calculate_q(bigrams_c, uni_cl, uni_cr, N):
  # returns table of pmi for bigrams

  q_k = dict()
  for pair,count in bigrams_c.items():
    w1, w2 = pair
    q_k[pair] = count/N * math.log(N * count / (uni_cl[w1] * uni_cr[w2]), 2)
  return q_k

In [5]:
def calculate_s(current_classes, q_k, uni_cl, uni_cr):
  # returns table of subtractions

  s_k = dict()
  for word in current_classes:
    s_k[word] = sum(q_k.get((wordL, word), 0) for wordL in uni_cl.keys()) \
    + sum(q_k.get((word, wordR), 0) for wordR in uni_cr.keys()) \
    - q_k.get((word, word), 0)
  return s_k


In [6]:
def compute_loss(current_classes, q_k, s_k, uni_cl, uni_cr, bigrams_c, N):
  # returns table of losses
  L_ab = dict()

  for i in range(len(current_classes)):
    for j in range(i+1, len(current_classes)):
      a,b = current_classes[i], current_classes[j]

      # sums computation
      sum_q_l_ab = 0
      uni_c_rab = uni_cr[a] + uni_cr[b]

      for wordL in uni_cl.keys():
        if wordL == a or wordL == b: continue
        bigr_c_lab = bigrams_c.get((wordL, a),0) + bigrams_c.get((wordL, b),0)

        if bigr_c_lab == 0: continue

        mi = bigr_c_lab/N * math.log(N * bigr_c_lab / (uni_cl[wordL] * uni_c_rab), 2)
        sum_q_l_ab += mi

      sum_q_ab_r = 0
      uni_c_lab = uni_cl[a] + uni_cl[b]

      for wordR in uni_cr.keys():
        if wordR == a or wordR == b: continue
        bigr_c_rab = bigrams_c.get((a, wordR),0) + bigrams_c.get((b, wordR),0)

        if bigr_c_rab == 0: continue

        mi = bigr_c_rab/N * math.log(N * bigr_c_rab / (uni_c_lab * uni_cr[wordR]), 2)
        sum_q_ab_r += mi

      # compute q_ab_ab
      bigr_c_abab = bigrams_c.get((a, b),0)  + bigrams_c.get((b, a),0) + bigrams_c.get((a, a),0) + bigrams_c.get((b, b),0)
      if bigr_c_abab == 0:
        q_ab_ab = 0
      else:
        q_ab_ab = bigr_c_abab/N * math.log(N * bigr_c_abab / (uni_c_lab * uni_c_rab), 2)

      L_ab[(a,b)] = s_k[a] + s_k[b] - q_k.get((a,b), 0) - q_k.get((b,a), 0) - q_ab_ab - sum_q_ab_r - sum_q_l_ab

  return L_ab

In [7]:
def merge_classes(best_a, best_b, classes, bigrams_c, uni_cl, uni_cr):

  # merge classes for a, b
  classes[best_a] = classes[best_a] + ' + ' + classes[best_b]
  del classes[best_b]

  # update unigram counts
  uni_cl[best_a] += uni_cl[best_b]
  del uni_cl[best_b]

  uni_cr[best_a] += uni_cr[best_b]
  del uni_cr[best_b]

  # update bigram counts
  updated_bigrams = bigrams_c.copy()

  for pair,count in bigrams_c.items():
    w1, w2 = pair
    if w1 == best_b and w2 == best_b:
      updated_bigrams[(best_a, best_a)] = bigrams_c.get((best_a, best_a),0) + count
      del updated_bigrams[(best_b, best_b)]

    if w1 == best_b and w2 != best_b:
      updated_bigrams[(best_a, w2)] = bigrams_c.get((best_a, w2),0) + count
      del updated_bigrams[(best_b, w2)]

    if w1 != best_b and w2 == best_b:
      updated_bigrams[(w1, best_a)] = bigrams_c.get((w1, best_a),0) + count
      del updated_bigrams[(w1, best_b)]

  return classes, updated_bigrams, uni_cl, uni_cr

In [8]:
def browns_clustering(words_to_cluster, name, limit):
  # initialise data structures
  history = dict()
  unigrams_c, bigrams_c = compute_counts(words_to_cluster)
  uni_cl = dict()
  uni_cr = dict()
  N = len(words_to_cluster)-1 # number of bigrams
  classes = dict()

  # compute ckl(), ckr()
  for word in unigrams_c.keys():
    uni_cl[word] = unigrams_c[word]
    uni_cr[word] = unigrams_c[word]

  # handle beginning and end
  uni_cl[words_to_cluster[-1]] -= 1
  uni_cr[words_to_cluster[0]] -= 1

  # init classes
  for word,count in unigrams_c.items():
    if count >= limit:
      classes[word] = word

  while len(classes) > 1:
    # compute q_k
    print('Computing q and s')
    q_k = calculate_q(bigrams_c, uni_cl, uni_cr, N)
    current_classes = list(classes.keys())

    # compute s_k
    s_k = calculate_s(current_classes, q_k, uni_cl, uni_cr)
    print(f'Computing loss table ..., actual mi {sum(q_k.values())}')

    # compute losses
    L_ab = compute_loss(current_classes, q_k, s_k, uni_cl, uni_cr, bigrams_c, N)

    # choose best pair to merge
    best_pair = min(L_ab, key=L_ab.get)
    best_a, best_b = best_pair
    print(f'Best pair to merge : {best_a}, {best_b} with loss : {L_ab[best_pair]}' )
    print('-' * 100 + '\n')

    # merge classes + update counts
    classes, bigrams_c, uni_cl, uni_cr = merge_classes(best_a, best_b, classes, bigrams_c, uni_cl, uni_cr)
    history[len(classes)] = best_pair
    print(f'Number of classes : {len(classes)}')

    # save 15 classes
    if len(classes) == 15:
      # Save dictionary
      with open(f'15_classes_{name}.txt', "w") as file:
        for key, value in classes.items():
            file.write(f"{key}: {value}\n")

  # save history
  with open(f'history_{name}.txt', "w") as file:
          file.write(f"number of classes: pair to merge\n")
          for key, value in history.items():
              file.write(f"{key}: {value}\n")




In [9]:
wordsEN, tagsEN= get_data(url3)
wordsCZ, tagsCZ= get_data(url4)

In [18]:
# perform clutering for english data
browns_clustering(wordsEN[:8000], 'EN', 10)

Computing q and s
Computing loss table ..., actual mi 4.996336755075334
Best pair to merge : subject, case with loss : 0.0021968411387180385
----------------------------------------------------------------------------------------------------

Number of classes : 111
Computing q and s
Computing loss table ..., actual mi 4.994139913936615
Best pair to merge : may, cannot with loss : 0.0026694731952487914
----------------------------------------------------------------------------------------------------

Number of classes : 110
Computing q and s
Computing loss table ..., actual mi 4.991470440741367
Best pair to merge : structure, individuals with loss : 0.0026751435455560704
----------------------------------------------------------------------------------------------------

Number of classes : 109
Computing q and s
Computing loss table ..., actual mi 4.9887952971958125
Best pair to merge : It, there with loss : 0.003479835349871169
-------------------------------------------------------

In [ ]:
# perform clusterin on czech data
browns_clustering(wordsCZ[:8000], 'CZ', 10)

Computing q and s
Computing loss table ..., actual mi 7.557964220781474
Best pair to merge : listopadu, OKD with loss : 0.0030832696793098354
----------------------------------------------------------------------------------------------------

Computing q and s
Computing loss table ..., actual mi 7.554880951102163
Best pair to merge : který, které with loss : 0.0033738533738386783
----------------------------------------------------------------------------------------------------

Computing q and s
Computing loss table ..., actual mi 7.551507097728323
Best pair to merge : J, státu with loss : 0.004025469551221374
----------------------------------------------------------------------------------------------------

Computing q and s
Computing loss table ..., actual mi 7.5474816281771
Best pair to merge : bude, musí with loss : 0.004422152935165135
----------------------------------------------------------------------------------------------------

Computing q and s
Computing loss table .

#3) Tags clustering

In [16]:
# perform clustering for ENG tags
browns_clustering(tagsEN,name = 'tagsEN', limit= 5)

Computing q and s
Computing loss table ..., actual mi 0.8833168327193148
Best pair to merge : RBR, WP$ with loss : 0.0002189398918106662
----------------------------------------------------------------------------------------------------

Number of classes : 35
Computing q and s
Computing loss table ..., actual mi 0.8830978928275041
Best pair to merge : JJR, RBR with loss : 0.00034888337197445853
----------------------------------------------------------------------------------------------------

Number of classes : 34
Computing q and s
Computing loss table ..., actual mi 0.8827490094555294
Best pair to merge : SYM, NNPS with loss : 0.0008193622508581421
----------------------------------------------------------------------------------------------------

Number of classes : 33
Computing q and s
Computing loss table ..., actual mi 0.881929647204671
Best pair to merge : PRP, EX with loss : 0.0010847100426949925
-----------------------------------------------------------------------------

In [17]:
# perform clustering for 70 000 CZ tags
browns_clustering(tagsCZ[:70000], name = 'tagsCZ', limit= 5)

Computing q and s
Computing loss table ..., actual mi 1.7993522739317134
Best pair to merge : CrFS6----------, PSFS6-P1------- with loss : 4.2807217565245725e-05
----------------------------------------------------------------------------------------------------

Number of classes : 454
Computing q and s
Computing loss table ..., actual mi 1.7993094667141478
Best pair to merge : PZXP6----------, Ca--6---------- with loss : 6.143881513639483e-05
----------------------------------------------------------------------------------------------------

Number of classes : 453
Computing q and s
Computing loss table ..., actual mi 1.7992480278990113
Best pair to merge : PSZS6FS3-------, PSZS6-P1------- with loss : 7.104686862486416e-05
----------------------------------------------------------------------------------------------------

Number of classes : 452
Computing q and s
Computing loss table ..., actual mi 1.7991769810303866
Best pair to merge : AUFS6M---------, CrFS6---------- with loss :